# Análise de Cluster

O conjunto de dados usado neste experimento é base da PNAE 2018 restruturada por cidade.

## Importando pacotes

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

## Carregando dados

In [3]:
base = pd.read_csv('PNAE_RECURSOS_REPASSADOS_2018_FINAL.csv')

In [4]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5513 entries, 0 to 5512
Data columns (total 13 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   CIDADE                                            5513 non-null   object 
 1   ATENDIMENTO EDUCACIONAL ESPECIALIZADO (AEE)       5513 non-null   float64
 2   CRECHE                                            5513 non-null   float64
 3   EJA                                               5513 non-null   float64
 4   ENSINO FUNDAMENTAL                                5513 non-null   float64
 5   ENSINO Mï¿½DIO                                    5513 non-null   float64
 6   ENSINO Mï¿½DIO INTEGRAL                           5513 non-null   float64
 7   INDï¿½GENA                                        5513 non-null   float64
 8   MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - FUNDAMENTAL  5513 non-null   float64
 9   MAIS EDUCAï¿½ï¿½O (

## Explorando dados

In [5]:
base.describe().T

,count,mean,std,min,25%,50%,75%,max
ATENDIMENTO EDUCACIONAL ESPECIALIZADO (AEE),"5,513.00","5,460.85","28,188.88",0.00,0.00,"1,060.00","3,710.00","1,102,930.00"
CRECHE,"5,513.00","91,768.77","415,035.60",0.00,"14,552.00","31,393.80","72,332.00","24,561,636.00"
EJA,"5,513.00","24,977.67","193,400.89",0.00,0.00,"3,584.00","14,784.00","6,104,498.00"
ENSINO FUNDAMENTAL,"5,513.00","304,055.66","1,505,341.77",0.00,"36,626.00","93,480.00","257,020.00","66,987,801.70"
ENSINO Mï¿½DIO,"5,513.00","63,772.59","1,019,313.88",0.00,0.00,0.00,0.00,"42,052,995.80"
ENSINO Mï¿½DIO INTEGRAL,"5,513.00","10,787.47","195,108.27",0.00,0.00,0.00,0.00,"7,917,920.00"
INDï¿½GENA,"5,513.00","5,446.85","61,433.02",0.00,0.00,0.00,0.00,"1,995,436.00"
MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - FUNDAMENTAL,"5,513.00","14,998.17","52,939.57",0.00,0.00,0.00,"1,022.40","1,549,560.80"
MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - INDï¿½GENA,"5,513.00",53.90,"1,435.82",0.00,0.00,0.00,0.00,"75,060.80"
MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - QUILOMBOLA,"5,513.00",207.77,"2,257.96",0.00,0.00,0.00,0.00,"96,457.60"


In [6]:
base['ESTADO']=base['CIDADE'].str.slice(0, 2)

In [7]:
# Intervalo interquartil e limites
def ver_outliers(df, feature):
    valor_iqr=iqr(df[feature],  keepdims=True)[0]

    l_sup = df[feature].mean()  + 1.5*valor_iqr
    l_inf = df[feature].mean()  - 1.5*valor_iqr

    print('Intervalo interquartil:', valor_iqr)
    print('Média', df[feature].mean())
    print('Limite Superior:', l_sup)
    print('Limite inferior:', l_inf)
    
    df[feature+'_OUT']=df[feature].apply(lambda x: 1 if (x<l_inf or x>l_sup) else 0)
    print()
    print(base.groupby(feature+'_OUT')[feature+'_OUT'].count())
    

In [8]:
ver_outliers(base, 'CRECHE')

Intervalo interquartil: 57780.0
Média 91768.77049156553
Limite Superior: 178438.77049156552
Limite inferior: 5098.770491565534

CRECHE_OUT
0    4499
1    1014
Name: CRECHE_OUT, dtype: int64


In [33]:
(base.groupby('CIDADE').sum()[['PRï¿½-ESCOLA']]).to_csv('PNAE_RECURSOS_REPASSADOS_2018_PRE.csv')

In [ ]:
campos = ['ATENDIMENTO EDUCACIONAL ESPECIALIZADO (AEE)',
'CRECHE',
'EJA',
'ENSINO FUNDAMENTAL',
'ENSINO Mï¿½DIO',
'ENSINO Mï¿½DIO INTEGRAL',
'INDï¿½GENA',
'MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - FUNDAMENTAL',
'MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - INDï¿½GENA',
'MAIS EDUCAï¿½ï¿½O (PARC. COMPLEM.) - QUILOMBOLA',
'PRï¿½-ESCOLA',
'QUILOMBOLA']

In [ ]:
base.groupby('ESTADO').mean()

In [ ]:
sns.set_theme(style="white")
corr =base.corr()
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(corr, vmin=-1, vmax=1, annot=True, cmap='RdBu')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12);
# save heatmap as .png file
# dpi - sets the resolution of the saved image in dots/inches
# bbox_inches - when set to 'tight' - does not allow the labels to be cropped
plt.savefig('heatmap.png', dpi=300, bbox_inches='tight')

In [ ]:
campos = ['CRECHE', 'ESTADO']

In [ ]:
X = base[campos]
X.boxplot(by='ESTADO')

In [ ]:
plt.scatter(X.iloc[0], X.iloc[1])
plt.grid()
plt.tight_layout()
plt.savefig('scatter.png', dpi=300)
plt.show()

In [ ]:
km = KMeans(n_clusters=5,init='k-means++',n_init=10,max_iter=300,random_state=21)

In [ ]:
#scaler = StandardScaler()
scaler = MinMaxScaler()

In [ ]:
X_std = scaler.fit_transform(X)
X_std

In [ ]:
y_km = km.fit_predict(X_std)

In [ ]:
print('Inércia: %.2f' % km.inertia_)

In [ ]:
inercia = []
for i in range(1, 16):
    km = KMeans(n_clusters=i, 
                init='k-means++', 
                n_init=10, 
                max_iter=300, 
                random_state=21)
    km.fit(X)
    inercia.append(km.inertia_)
plt.plot(range(1, 16), inercia, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inércia')
plt.tight_layout()
#plt.savefig('images/11_03.png', dpi=300)
plt.show()

In [ ]:
import numpy as np
from matplotlib import cm
from sklearn.metrics import silhouette_samples

km = KMeans(n_clusters=7, 
            init='k-means++', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=21)
y_km = km.fit_predict(X)

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]
silhouette_vals = silhouette_samples(X, y_km, metric='euclidean')
y_ax_lower, y_ax_upper = 0, 0
yticks = []
for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')

plt.tight_layout()
plt.savefig('11_04.png', dpi=300)
plt.show()